In [5]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

train_data_path = 'data/train/'
test_data_path = 'data/test/'

def write_gap(MODEL, image_size, module_name, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_data_path, image_size, shuffle=False, batch_size=16)
    #test_generator = gen.flow_from_directory(test_data_path, image_size, shuffle=False, batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator)
    #test = model.predict_generator(test_generator, test_generator.nb_sample)
    with h5py.File("bottleneck_features/bottleneck_v3.h5") as h:
        h.create_dataset("train", data=train)
        #h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

#write_gap(ResNet50, (224, 224), 'ResNet50')
write_gap(InceptionV3, (299, 299), 'InceptionV3', inception_v3.preprocess_input)
#write_gap(Xception, (299, 299), 'Xception', xception.preprocess_input)

Found 25000 images belonging to 2 classes.


KeyboardInterrupt: 

In [4]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *

np.random.seed(2017)

X_train = []
X_test = []

#for filename in ["bottleneck_features/gap_ResNet50.h5", "bottleneck_features/gap_Xception.h5", "bottleneck_features/gap_InceptionV3.h5"]:
for filename in ["bottleneck_features/gap_ResNet50.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

np.random.seed(2017)

print(X_train.shape[1:])

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


#model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

(2048,)


In [ ]:
model.predict()